In [ ]:
import pandas as pd
from safeaipackage import core

from config import MAIN_MODELS, MODEL_ORDER
from src.load_data import load_predictions

In [ ]:
df = load_predictions()
df = df[df.model.isin(MAIN_MODELS)].copy()
model_order = [m for m in MODEL_ORDER if m in df.model.unique()]
df["model"] = pd.Categorical(df["model"], categories=model_order, ordered=True)
df = df[df.age_group == "00+"]
df = df[df["type"] == "median"]

# Compute RGA for each model and horizon
results = []
for h in [1, 2, 3, 4]:
    df_h = df[df.horizon == h]
    for m in df_h.model.unique():
        df_temp = df_h[df_h.model == m]
        score = core.rga(df_temp.value, df_temp.truth)
        results.append({"model": m, "horizon": h, "rga": score})

df_results = pd.DataFrame(results)
df_results["model"] = pd.Categorical(
    df_results["model"], categories=model_order, ordered=True
)
df_results = df_results.sort_values(by=["horizon", "model"]).reset_index(drop=True)

df_wide = df_results.pivot(index="model", columns="horizon", values="rga")
df_wide.columns = [f"horizon_{h}" for h in df_wide.columns]
df_wide = df_wide.reset_index()

In [ ]:
df_wide

### Generate LaTeX table code


In [ ]:
latex_table = df_wide.to_latex(
    float_format="%.3f",  # format floats to 3 decimal places
    caption="RGA scores by model and forecast horizon",
    label="tab:rga_scores",
    column_format="lcccc",
    escape=False,
    index=False
)

print(latex_table)